# OpenFermion Gran Canonical

A diferencia de FermionicML, donde se trabaja en el canónico, la idea es implementar las matrices densidad de 1 y 2 cuerpos en el ensamble canónico, utilizando para ello operadores dados por OpenFermion, por la facilidad de generar los operadores (rhoKarrays) tomando la get_sparse_operator sobre la base. 

In [135]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)
    
    def idx_to_repr(self, idx):
        return self.canonicals[idx]
    
    def opr_to_idx(self, opr):
        return self.base.index(opr)

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    oper = self.bin_to_op(b)
                    basis.append(oper)
                    num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None):
        self.d = d
        self.num = num
        self.base, self.num_ele = self.create_basis(d, num)
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        
    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat
        
# Calculo de generadores de rho1
def rho_1_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            op = basis.cdc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    return np.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)

# Calculo de generadores de K (usado para quasiparticles)
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis):
    # La entrada i, j contiene C_j^\dag C_i    i, j \in t_basis
    mat = np.zeros((t_basis.size, t_basis.size, basis.size, basis.size))
    for i in tqdm(range(t_basis.size)):
        for j in range(t_basis.size):
            op = t_basis.base[j]*of.utils.hermitian_conjugated(t_basis.base[i])
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]

# Calculo de rho2 (via generadores) de un vector en la base canonica
def rho_2(vect, rho_2_arrays):
    return np.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)

# Calculo la matrix rho de cuasipartículas
def rho_qsp(vect, rho_1_arrays, k_arrays):
    rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
@functools.lru_cache(maxsize=None) 
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

@functools.lru_cache(maxsize=None) 
def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)



Una vez definidos los operados globales, definamos funciones auxiliares utilizadas para el problema

In [ ]:
def rand_state(d, parity = True):
    st = np.random.uniform(0,1,2**d)
    if parity:
        st[parity_levels(d)] = 0
    
    return st/np.linalg.norm(st)

In [142]:
# PRUEBITAS

s_eig = lambda m: np.real(np.sort(np.linalg.eigvals(m))[::-1])

d = 4
est = rand_state(d)
r1 = rho_1(est, rho_1_arrays)
rqsp = rho_qsp(est, rho_1_arrays, k_arrays)

p = r1[1,1]
q = 1-p

est_pm, est_pm_comp = measure(basis, est)


r1_pm = rho_1(est_pm, rho_1_arrays)
r1_pm_comp = rho_1(est_pm_comp, rho_1_arrays)
print(-np.cumsum(s_eig(r1))+p*np.cumsum(s_eig(r1_pm))+q*np.cumsum(s_eig(r1_pm_comp)))

# Lo mismo para QSP
rqsp_pm = rho_qsp(est_pm, rho_1_arrays, k_arrays)
rqsp_pm_comp = rho_qsp(est_pm_comp, rho_1_arrays, k_arrays)
print(-np.cumsum(s_eig(rqsp))+p*np.cumsum(s_eig(rqsp_pm))+q*np.cumsum(s_eig(rqsp_pm_comp)))

[ 5.89524829e-02  1.17904966e-01  5.89524829e-02 -2.22044605e-16]
[ 3.89160474e-02  7.78320949e-02  1.16748142e-01  1.55664190e-01
  1.16748142e-01  7.78320949e-02  3.89160474e-02 -4.44089210e-16]


In [90]:
d = 4 # número de niveles
num = None # número de particulas fijo, None es GC

basis = fixed_basis(d)
rho_1_arrays = rho_1_gen(basis)
k_arrays = k_gen(basis)

100%|██████████| 4/4 [00:00<00:00, 40.37it/s]


In [116]:
#print(k_arrays[0,0])
idx = 1, 8
print(basis.base[6], basis.base[-1])
x = basis.canonicals[6] + basis.canonicals[-1]
#print(basis.base)
rr=rho_qsp(x, rho_1_arrays, k_arrays)
np.linalg.eigvals(rr)

1.0 [1^ 2^] 1.0 [0^ 1^ 2^ 3^]


array([ 1.61803399, -0.61803399, -0.61803399,  1.61803399,  2.        ,
        2.        , -1.        , -1.        ])

In [41]:
# CONFIG
d = 4 # número de niveles
num = None # número de particulas fijo, None es GC

basis = fixed_basis(d)
rho_1_arrays = rho_1_gen(basis)
k_arrays = k_gen(basis)
#t_basis = fixed_basis(d, num=2)
#rho_2_arrays = rho_2_gen(basis, t_basis)

100%|██████████| 4/4 [00:00<00:00, 45.11it/s]


In [10]:
d=3
t_basis = fixed_basis(d, num=2)
basis = fixed_basis(d)
#print(t_basis.base[1])

mat = np.zeros((t_basis.size, t_basis.size, basis.size, basis.size))
for i in range(t_basis.size):
    for j in range(t_basis.size):
        op = t_basis.base[j]*of.utils.hermitian_conjugated(t_basis.base[i])
        if basis.num == None:
            mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
        else:
            mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]

print(t_basis.base)
print(basis.base)
mat[1,0]

[1.0 [1^ 2^], 1.0 [0^ 2^], 1.0 [0^ 1^]]
[1.0 [], 1.0 [2^], 1.0 [1^], 1.0 [1^ 2^], 1.0 [0^], 1.0 [0^ 2^], 1.0 [0^ 1^], 1.0 [0^ 1^ 2^]]


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [67]:
# BENCHMARK RHO1
d = 4
b = fixed_basis(d)
vect = b.canonicals[3] + b.canonicals[5]
print(b.base[3], b.base[5])
ra = rho_1_gen(b)
rho_1(vect, ra)


1.0 [2^ 3^] 1.0 [1^ 3^]


100%|██████████| 4/4 [00:00<00:00, 31.74it/s]


array([[0., 0., 0., 0.],
       [0., 1., 1., 0.],
       [0., 1., 1., 0.],
       [0., 0., 0., 2.]])

In [43]:
est = 1
c = b.canonicals[est]
mat = np.zeros((b.d,b.d))
for i in range(b.d):
    for j in range(b.d):
        mat[i, j] = c.T @ ra[i,j,::] @ c

print(mat)
np.einsum('k,ijkl,l->ij', c, ra, c)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.]])

In [35]:
b = fixed_basis(3)
v = b.base[1]
print(b.base)
print(v)
print(b.rho_1(v))

ra = rho_1_gen(b)
rho_1(b.canonicals[1], ra)


[1.0 [], 1.0 [2^], 1.0 [1^], 1.0 [1^ 2^], 1.0 [0^], 1.0 [0^ 2^], 1.0 [0^ 1^], 1.0 [0^ 1^ 2^]]
1.0 [2^]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]


100%|██████████| 3/3 [00:00<00:00, 69.91it/s]


array([[0., 0., 0.],
       [0., 0., 0.],
       [1., 1., 1.]])

In [ ]:
t_basis = fixed_basis(d, num=2)

mat = np.zeros(t_basis.size, t_basis.size, basis.size, basis.size)

In [3]:
basis = fixed_basis(d)
print(basis.base)
print(basis.num_ele)
rho_1_gen(basis)[0,1].shape #C_dag(j) c_i

[1.0 [], 1.0 [3^], 1.0 [2^], 1.0 [2^ 3^], 1.0 [1^], 1.0 [1^ 3^], 1.0 [1^ 2^], 1.0 [1^ 2^ 3^], 1.0 [0^], 1.0 [0^ 3^], 1.0 [0^ 2^], 1.0 [0^ 2^ 3^], 1.0 [0^ 1^], 1.0 [0^ 1^ 3^], 1.0 [0^ 1^ 2^], 1.0 [0^ 1^ 2^ 3^]]
[]


100%|██████████| 4/4 [00:00<00:00, 36.03it/s]


(16, 16)

In [ ]:

#bbb = fixed_basis(d).rho_1_arrays
i,j = 1, 1
base = fixed_basis(d).base
cano = fixed_basis(d).canonicals
OPNUM = 5
wfc = base[OPNUM]
idx = cano[OPNUM]
print(idx, wfc)
OPME = fixed_basis.cdc(j, i)
print(of.get_sparse_operator(OPME, n_qubits=d))

#print(wfc)
mean_val = lambda x: np.real(np.transpose(idx) @ of.get_sparse_operator(x, n_qubits=d) @ idx)

print(mean_val(fixed_basis.bdb(j, i)))
#print(fixed_basis.bdb(j, i, d))
#print(op.get_sparse_operator(fixed_basis.bdb(j, i, d), n_qubits=d))
#op.linalg.expectation(fixed_basis.bdb(j, i, d), state=op.get_sparse_operator(base[1], n_qubits=d))
print(base[6], base[5])
print(fixed_basis(d).mean_val(wfc, OPME))